Create a table for experimentation with odd number of entries. It has just prices and units. We also add some sample data.

In [73]:
DROP TABLE IF EXISTS MY_EXPERIMENT;

CREATE TABLE MY_EXPERIMENT (
    price INT
);

insert into MY_EXPERIMENT(price) values (30);
insert into MY_EXPERIMENT(price) values (10);
insert into MY_EXPERIMENT(price) values (20);
insert into MY_EXPERIMENT(price) values (10);
insert into MY_EXPERIMENT(price) values (30);

select * FROM MY_EXPERIMENT;

Commands completed successfully

Commands completed successfully

Commands completed successfully

Commands completed successfully

Commands completed successfully

Commands completed successfully

Commands completed successfully

(5 row(s) affected)

Total execution time: 00:00:00.030

price
30
10
20
10
30


Attempt the common trick for median

In [74]:
select price,  (2 * row_number() over (ORDER BY price asc) - count(*) over ()) as indicator from my_experiment;

(5 row(s) affected)

Total execution time: 00:00:00.010

price,indicator
10,-3
10,-1
20,1
30,3
30,5


Create a table for experimentation as above but with even number of entries. And reattempt the trick.

In [75]:
truncate MY_EXPERIMENT;

insert into MY_EXPERIMENT(price) values (30);
insert into MY_EXPERIMENT(price) values (10);
insert into MY_EXPERIMENT(price) values (20);
insert into MY_EXPERIMENT(price) values (10);
insert into MY_EXPERIMENT(price) values (30);
insert into MY_EXPERIMENT(price) values (15);

select price, (2 * row_number() over (ORDER BY price asc) - count(*) over ()) as indicator from my_experiment;

Commands completed successfully

Commands completed successfully

Commands completed successfully

Commands completed successfully

Commands completed successfully

Commands completed successfully

Commands completed successfully

(6 row(s) affected)

Total execution time: 00:00:00.027

price,indicator
10,-4
10,-2
15,0
20,2
30,4
30,6


Our custom made median lower

In [76]:
-- taken from
-- https://weblogs.sqlteam.com/peterl/2009/09/16/median-and-weighted-median/
-- best

CREATE OR REPLACE FUNCTION peso_median() RETURNS FLOAT AS $$
DECLARE my_result FLOAT;
BEGIN
select avg(price) into my_result from (
    select price, (2 * row_number() over (ORDER BY price asc) - count(*) over ()) as indicator from my_experiment
) f where f.indicator between 0 and 2;

RETURN my_result;        
END;
$$ 
STABLE
LANGUAGE plpgsql;

-- inspired from https://en.wikipedia.org/wiki/Percentile#The_nearest-rank_method
-- implemented in python baseline
--
CREATE OR REPLACE FUNCTION nearest_rank(threshold FLOAT) RETURNS FLOAT AS $$
DECLARE my_result FLOAT;
BEGIN
select price into my_result from (
 select price, ( (count(*) over (ORDER BY price asc))::real / (count(*) over ()) ) as xxx from my_experiment
) f where xxx >= threshold limit 1;
RETURN my_result;
END;
$$ 
STABLE
LANGUAGE plpgsql;

Commands completed successfully

Commands completed successfully

Total execution time: 00:00:00.012

Median when we have even elements

In [77]:
select * from peso_median();

(1 row(s) affected)

Total execution time: 00:00:00.010

peso_median
17.5


Median when we have odd elements

In [78]:
truncate MY_EXPERIMENT;

insert into MY_EXPERIMENT(price) values (30);
insert into MY_EXPERIMENT(price) values (10);
insert into MY_EXPERIMENT(price) values (20);
insert into MY_EXPERIMENT(price) values (10);
insert into MY_EXPERIMENT(price) values (30);

select * from peso_median();

Commands completed successfully

Commands completed successfully

Commands completed successfully

Commands completed successfully

Commands completed successfully

Commands completed successfully

(1 row(s) affected)

Total execution time: 00:00:00.018

peso_median
20.0


Peso calculation (https://weblogs.sqlteam.com/peterl/2009/09/16/median-and-weighted-median/)

In [79]:
truncate MY_EXPERIMENT;

insert into MY_EXPERIMENT(price) values (30);
insert into MY_EXPERIMENT(price) values (10);
insert into MY_EXPERIMENT(price) values (20);
insert into MY_EXPERIMENT(price) values (10);
insert into MY_EXPERIMENT(price) values (30);
insert into MY_EXPERIMENT(price) values (30);

select * from MY_EXPERIMENT;

Commands completed successfully

Commands completed successfully

Commands completed successfully

Commands completed successfully

Commands completed successfully

Commands completed successfully

Commands completed successfully

(6 row(s) affected)

Total execution time: 00:00:00.020

price
30
10
20
10
30
30


In [80]:
-- taken from
-- https://weblogs.sqlteam.com/peterl/2009/09/16/median-and-weighted-median/
-- best

CREATE OR REPLACE FUNCTION peso_occurences_median() RETURNS real AS $$
DECLARE my_result real;
BEGIN
        SELECT  SUM(partition_size_times_price::real) / SUM(partition_cum_size::real) into my_result
        FROM    (
                SELECT  SUM(price) OVER (PARTITION BY price) AS partition_size_times_price,
                        2 * ROW_NUMBER() OVER (ORDER BY price) - COUNT(*) OVER () AS xxx,
                        COUNT(*) OVER (PARTITION BY price) AS partition_cum_size
                FROM    MY_EXPERIMENT
                ) AS d
        WHERE   xxx BETWEEN 0 AND 2;
RETURN my_result;        
END;
$$ 
STABLE
LANGUAGE plpgsql;

select * from peso_occurences_median()

Commands completed successfully

(1 row(s) affected)

Total execution time: 00:00:00.013

peso_occurences_median
27.5


<span style="color: rgb(0, 0, 0); font-family: -apple-system, BlinkMacSystemFont, sans-serif;">The peso median now fails completely misarbly. not as good as he claims huh?</span>

In [81]:
select * from peso_median()

(1 row(s) affected)

Total execution time: 00:00:00.010

peso_median
25.0


For comparison the nearest\_rank

In [82]:
select * from nearest_rank(0.5)

(1 row(s) affected)

Total execution time: 00:00:00.011

nearest_rank
20.0


Weighted nearest rank for the occurence table. Hopefully they are the same.

In [83]:
DROP TABLE IF EXISTS MY_OCCURENCES_EXPERIMENT;

CREATE TABLE MY_OCCURENCES_EXPERIMENT AS (

select price, count(*) as occurences from MY_EXPERIMENT

group by price

);

select * FROM MY_occurences_EXPERIMENT;

Commands completed successfully

Commands completed successfully

(3 row(s) affected)

Total execution time: 00:00:00.014

price,occurences
30,3
10,2
20,1


In [84]:
select price from (
 select price, ( (count(*) over (ORDER BY price asc))::real / (count(*) over ()) ) as xxx from my_occurences_experiment
) f where xxx >= 0.5 limit 1;

(1 row(s) affected)

Total execution time: 00:00:00.008

price
20


See how they are the sme. We are on the right track.

Weighted experiment

In [87]:
DROP TABLE IF EXISTS MY_WEIGHTED_EXPERIMENT;

CREATE TABLE MY_WEIGHTED_EXPERIMENT (
    price REAL,
    units REAL
);

insert into MY_WEIGHTED_EXPERIMENT(price, units) values (30.5, 5);
insert into MY_WEIGHTED_EXPERIMENT(price, units) values (10.5, 20);
insert into MY_WEIGHTED_EXPERIMENT(price, units) values (20.5, 10);
insert into MY_WEIGHTED_EXPERIMENT(price, units) values (10.5, 20);
insert into MY_WEIGHTED_EXPERIMENT(price, units) values (30.5, 5);

select * FROM MY_WEIGHTED_EXPERIMENT;

Commands completed successfully

Commands completed successfully

Commands completed successfully

Commands completed successfully

Commands completed successfully

Commands completed successfully

Commands completed successfully

(5 row(s) affected)

Total execution time: 00:00:00.027

price,units
30.5,5.0
10.5,20.0
20.5,10.0
10.5,20.0
30.5,5.0


Closest rank for weighted

In [90]:
drop function my_weighted_nearest_rank;
CREATE OR REPLACE FUNCTION my_weighted_nearest_rank(threshold FLOAT) RETURNS real AS $$
DECLARE my_result real;
BEGIN
select price into my_result from (
 select price, units, (sum(units) over (ORDER BY price asc) / sum(units) over () ) as xxx from MY_WEIGHTED_EXPERIMENT
) f where xxx >= threshold limit 1;
RETURN my_result;        
END;
$$ 
STABLE
LANGUAGE plpgsql;

Commands completed successfully

Commands completed successfully

Total execution time: 00:00:00.006

In [91]:
select * from my_weighted_nearest_rank(0.75);

(1 row(s) affected)

Total execution time: 00:00:00.012

my_weighted_nearest_rank
20.5


Partitioned example

In [67]:
-- inspired from
-- https://stackoverflow.com/questions/49077403/postgres-arg-min-in-a-group-by
-- 

DROP TABLE IF EXISTS MY_PARTITIONED_EXPERIMENT;

CREATE TABLE MY_PARTITIONED_EXPERIMENT (
    price REAL,
    units REAL,
    region INT
);

insert into MY_PARTITIONED_EXPERIMENT(price, units, region) values (30, 5, 1);
insert into MY_PARTITIONED_EXPERIMENT(price, units, region) values (10, 20, 1);
insert into MY_PARTITIONED_EXPERIMENT(price, units, region) values (20, 10, 1);
insert into MY_PARTITIONED_EXPERIMENT(price, units, region) values (10, 20, 1);
insert into MY_PARTITIONED_EXPERIMENT(price, units, region) values (30, 5, 1);

insert into MY_PARTITIONED_EXPERIMENT(price, units, region) values (300, 10, 2);
insert into MY_PARTITIONED_EXPERIMENT(price, units, region) values (100, 200, 2);
insert into MY_PARTITIONED_EXPERIMENT(price, units, region) values (200, 100, 2);
insert into MY_PARTITIONED_EXPERIMENT(price, units, region) values (100, 200, 2);
insert into MY_PARTITIONED_EXPERIMENT(price, units, region) values (300, 50, 2);

select * FROM MY_PARTITIONED_EXPERIMENT;


Commands completed successfully

Commands completed successfully

Commands completed successfully

Commands completed successfully

Commands completed successfully

...

Commands completed successfully

Commands completed successfully

(10 row(s) affected)

Total execution time: 00:00:00.030

price,units,region
30.0,5.0,1
10.0,20.0,1
20.0,10.0,1
10.0,20.0,1
30.0,5.0,1
300.0,10.0,2
100.0,200.0,2
200.0,100.0,2
100.0,200.0,2
300.0,50.0,2


Selections based on partitions and threshold 0.7

In [68]:
drop function my_partitioned_weighted_nearest_rank;
CREATE OR REPLACE FUNCTION my_partitioned_weighted_nearest_rank(threshold FLOAT) RETURNS TABLE(theregion int, theprice real) 
AS $$
begin
    return query 
        select distinct on (region) region, price
        from (
            select 
                price, 
                units, 
                region, 
                (sum(units) over (partition by region order by price asc) / sum(units)  over (partition by region)) as xxx
            from MY_PARTITIONED_EXPERIMENT
        ) f 
        where f.xxx >= threshold
        order by region;
end;
$$ 
STABLE
LANGUAGE plpgsql;

select * from my_partitioned_weighted_nearest_rank(0.7);

Commands completed successfully

Commands completed successfully

(2 row(s) affected)

Total execution time: 00:00:00.013

theregion,theprice
1,20.0
2,100.0


Verification

In [69]:
truncate my_weighted_experiment;

insert into my_weighted_experiment
select price, units from my_partitioned_experiment
where region = 1;

select * from my_weighted_nearest_rank(0.7);

Commands completed successfully

Commands completed successfully

(1 row(s) affected)

Total execution time: 00:00:00.028

my_weighted_nearest_rank
20.0


In [70]:
truncate my_weighted_experiment;

insert into my_weighted_experiment
select price, units from my_partitioned_experiment
where region = 2;

select * from my_weighted_nearest_rank(0.7);

Commands completed successfully

Commands completed successfully

(1 row(s) affected)

Total execution time: 00:00:00.012

my_weighted_nearest_rank
100.0
